# Webscraper with BeautifulSoup

In [ ]:
import requests
from bs4 import BeautifulSoup
import html2json
import json
from jsonpath_ng import jsonpath, parse
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from seleniumwire import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.common.by import By

class scraper:
    def __init__(self, url):
        self.url = 'https://www.ulta.com/shop/makeup/face'
        self.json = {}

    
    def get_content(self):
        r = requests.get(self.url)
        return r.content
    
    def get_category(self):
        r = requests.get(self.url)
        soup = BeautifulSoup(r.content, "html.parser")
        content = soup.find_all(class_='CategoryCard')
        categories = {}
        for c in content:
            category_tag = c.find('a', class_='Link_Huge')
            category_name = category_tag.get_text()
            category_link = category_tag.get('href')
            self.json[category_name] = {}
            self.json[category_name]['url'] = category_link
            self.json[category_name]['products'] = []

            get_product_links(category_link)
        

    
    def get_product_links(self):
        soup = BeautifulSoup(self.content, "html.parser")
        content = soup.find_all(class_='ProductCard')
        products = {}

        for c in content:
            product_link_tag = c.find('a', class_='Link_Huge Link_Huge--secondary')
            product_link = product_link_tag.get('href')
            self.json[category_name]['products'].append(product_link)
            get_product_details(product_link)
    
    def get_product_details(product_link):
        r = requests.get(product_link)
        soup = BeautifulSoup(r.content, "html.parser")
        html_content = soup.find_all(class_='ProductHero__content')
        details = ''

        def extract_text_fields(json_data):
            """Extract all text fields using jsonpath-ng."""
            jsonpath_expression = parse('$.._value')
            matches = jsonpath_expression.find(json_data)
            return [match.value for match in matches]

        def html_to_text_json(html_content):
            """Convert HTML content to a nested JSON containing only text fields."""
            json_data = html_to_json.convert(html_content)
            text_fields = extract_text_fields(json_data)
            #print(json.dumps(json_data, indent=4))
            return text_fields
        
        # Convert HTML to text-only JSON
        text_json = html_to_text_json(str(html_content))
        
        # Print the JSON output
        #print(json.dumps(text_json, indent=4) )
        details = ' '.join(text_json)
        get_spec_product_details(product_link)
        get_reviews(product_link)


    def get_spec_prod_details(product_link): 
        r = requests.get(product_link)
        soup = BeautifulSoup(r.content, "html.parser")
        content = soup.find_all(class_='ProductHero__content')
    
        for c in content:
            name = c.find(class_= 'Text-ds Text-ds--body-1 Text-ds--left')
            price = c.find(class_='ProductPricing')
            id_num = c.find(class_='Text-ds Text-ds--body-3 Text-ds--left Text-ds--neutral-600')

            product_price = price.get_text()
            product_name = name.get_text()
            product_id = id_num.get_text()

            self.json[category_name]['products'] = {
                'name': product_name,
                'price': product_price,
                'id': product_id
            }
    

            
    def get_product_reviews(product_link):
        options = webdriver.ChromeOptions()
        # Uncomment the following line to run in headless mode
        #options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
          
        # Initialize the WebDriver
        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
        driver.maximize_window()
        driver.get(product_link)
        xhr_links = [] #stores the specific xhr links needed
          
        # Function to extract the XHR URL
        def extract_xhr_url():
            for request in driver.requests: 
                xhr_links.append(request.url)    
         
        # Define a function to gradually scroll through the entire page
        def slow_scroll(start_position):
            scroll_height = driver.execute_script('return document.body.scrollHeight')
            current_position = start_position
            while current_position < scroll_height:
                driver.execute_script('window.scrollTo(0, {});'.format(current_position))
                current_position += 100  # Adjust scrolling speed by changing increment
                time.sleep(0.2)  # Adjust sleep time to control scrolling speed
        
        try:
            # Wait for the section to be visible (adjust timeout as needed)
            section_id = "reviews"
            section_element = WebDriverWait(driver, 20).until(
                EC.visibility_of_element_located((By.ID, section_id))
            )
            height =  0
            time.sleep(2)
            slow_scroll(height)
            extract_xhr_url()
        finally:
            # Close the browser
            driver.quit()
            
 

# Scratch Work 

In [ ]:
import requests
from bs4 import BeautifulSoup
import html_to_json
import json
from jsonpath_ng import jsonpath, parse


from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from seleniumwire import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.common.by import By

import requests
from bs4 import BeautifulSoup
import html_to_json
import json
from jsonpath_ng import jsonpath, parse




    
def get_category(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, "html.parser")
    content = soup.find_all(class_='CategoryCard')
    categories = {}
    #print(content)
    for c in content:
        category_tag = c.find('a', class_='pal-c-Link pal-c-Link--primary pal-c-Link--default')
        category_name = category_tag.get_text()
        category_link = category_tag.get('href')
        product_info[category_name] = {}
        product_info[category_name]['url'] = category_link
        product_info[category_name]['products'] = []
        #print(category_link)
        get_product_links(category_link, category_name)


   

def get_product_links(category_link, category_name):
    r = requests.get(category_link)
    soup = BeautifulSoup(r.content, "html.parser")
    content = soup.find_all(class_='ProductCard')
    products = {}
    products['name'] = category_name
    products['url'] = category_link
    products['product_list'] = []
    
    
    for c in content:
        product_link_tag = c.find('a', class_='pal-c-Link pal-c-Link--primary pal-c-Link--default')
        product_link = product_link_tag.get('href')
        product_info[category_name]['products'].append(product_link)
        products['product_list'].append(get_product_details(product_link))
    print(products)

def get_product_details(product_link):
    r = requests.get(product_link)
    soup = BeautifulSoup(r.content, "html.parser")
    html_content = soup.find_all(class_='ProductHero__content')
    details = ''
    all_products = {}
    all_products['url'] = product_link
    def extract_text_fields(json_data):
        jsonpath_expression = parse('$.._value')
        matches = jsonpath_expression.find(json_data)
        return [match.value for match in matches]

        
    def html_to_text_json(html_content):
        """Convert HTML content to a nested JSON containing only text fields."""
        json_data = html_to_json.convert(html_content)
        text_fields = extract_text_fields(json_data)
        #print(json.dumps(json_data, indent=4))
        return text_fields
    
    # Convert HTML to text-only JSON
    text_json = html_to_text_json(str(html_content))
    
    # Print the JSON output
    #print(json.dumps(text_json, indent=4) )
    details = ' '.join(text_json)
    all_products['details'] = details
    return all_products

    #Next set of info to get
    #get_spec_product_details(product_link)
    #get_reviews(product_link)


def get_spec_prod_details(product_link): 
    r = requests.get(product_link)
    soup = BeautifulSoup(r.content, "html.parser")
    content = soup.find_all(class_='ProductHero__content')

    for c in content:
        name = c.find(class_= 'Text-ds Text-ds--body-1 Text-ds--left')
        price = c.find(class_='ProductPricing')
        id_num = c.find(class_='Text-ds Text-ds--body-3 Text-ds--left Text-ds--neutral-600')

        product_price = price.get_text()
        product_name = name.get_text()
        product_id = id_num.get_text()

        json[category_name]['products'] = {
            'name': product_name,
            'price': product_price,
            'id': product_id
        }
        
def get_product_reviews(product_link):
    options = webdriver.ChromeOptions()
    # Uncomment the following line to run in headless mode
    #options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
      
    # Initialize the WebDriver
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    driver.maximize_window()
    driver.get(product_link)
    xhr_links = [] #stores the specific xhr links needed
      
    # Function to extract the XHR URL
    def extract_xhr_url():
        for request in driver.requests: 
            xhr_links.append(request.url)    
     
    # Define a function to gradually scroll through the entire page
    def slow_scroll(start_position):
        scroll_height = driver.execute_script('return document.body.scrollHeight')
        current_position = start_position
        while current_position < scroll_height:
            driver.execute_script('window.scrollTo(0, {});'.format(current_position))
            current_position += 100  # Adjust scrolling speed by changing increment
            time.sleep(0.2)  # Adjust sleep time to control scrolling speed
    
    try:
        # Wait for the section to be visible (adjust timeout as needed)
        section_id = "reviews"
        section_element = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.ID, section_id))
        )
        height =  0
        time.sleep(2)
        slow_scroll(height)
        extract_xhr_url()
    finally:
        # Close the browser
        driver.quit()

url = 'https://www.ulta.com/shop/makeup/face'
product_info = {}
get_category(url) 

 

In [ ]:
import requests
from bs4 import BeautifulSoup
import html_to_json
import json
from jsonpath_ng import jsonpath, parse

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from seleniumwire import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.common.by import By


##Handle Pagination for Product Page to load more products
##Have to click the load more button

json = {}
categories = {}
ulta_link = 'https://www.ulta.com/shop/makeup/face'

#Get all categories: 
def get_category(self):
    r = requests.get(self.url)
    soup = BeautifulSoup(r.content, "html.parser")
    content = soup.find_all(class_='CategoryCard')
    categories = {}
    for c in content:
        category_tag = c.find('a', class_='Link_Huge')
        category_name = category_tag.get_text()
        category_link = category_tag.get('href')
        categories[category_name] = {}
        categories[category_name]['url'] = category_link
        categories[category_name]['products'] = ['this ', 'is', 'SPARTA']



## Extract Product Info to One String

In [ ]:
import requests
from bs4 import BeautifulSoup
import html_to_json
import json
from jsonpath_ng import jsonpath, parse

#<a class="Link_Huge Link_Huge--secondary" target="_self" href="https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420"><div class="ProductCard__image ProductCard__image__full__small"><div class="Image"><img class="" alt="Estée Lauder Double Wear Stay-in-Place Foundation" aria-hidden="true" loading="lazy" width="360" height="360" src="https://media.ulta.com/i/ulta/2309420?w=360&amp;$ProductCardNeutralBGLight$&amp;fmt=auto"></div><span class="ProductCard__variant"><p class="Text-ds Text-ds--body-3 Text-ds--left Text-ds--neutral-600">56 colors</p></span></div><div class="ProductCard__content"><h3 class="ProductCard__heading"><span class="ProductCard__brand"><span class="Text-ds Text-ds--body-2 Text-ds--left Text-ds--neutral-600">Estée Lauder</span></span><span class="ProductCard__product"><span class="Text-ds Text-ds--body-2 Text-ds--left">Double Wear Stay-in-Place Foundation</span></span></h3><div class="ProductCard__rating"><div class="ReviewStarsCard"><svg aria-hidden="true" width="12" height="12"><path d="M5.999 1c.136 0 .262.076.324.198L7.81 4.144l2.862.284a.358.358 0 01.222.61L8.54 7.373l.873 3.172a.361.361 0 01-.31.452l-.051.002a.36.36 0 01-.15-.039L5.998 9.524 3.1 10.959a.361.361 0 01-.099.033l-.05.005a.361.361 0 01-.362-.454l.873-3.172-2.357-2.335a.358.358 0 01.222-.61l2.862-.284 1.485-2.944A.364.364 0 015.999 1z" fill="currentColor" fill-rule="nonzero"></path></svg><svg aria-hidden="true" width="12" height="12"><path d="M5.999 1c.136 0 .262.076.324.198L7.81 4.144l2.862.284a.358.358 0 01.222.61L8.54 7.373l.873 3.172a.361.361 0 01-.31.452l-.051.002a.36.36 0 01-.15-.039L5.998 9.524 3.1 10.959a.361.361 0 01-.099.033l-.05.005a.361.361 0 01-.362-.454l.873-3.172-2.357-2.335a.358.358 0 01.222-.61l2.862-.284 1.485-2.944A.364.364 0 015.999 1z" fill="currentColor" fill-rule="nonzero"></path></svg><svg aria-hidden="true" width="12" height="12"><path d="M5.999 1c.136 0 .262.076.324.198L7.81 4.144l2.862.284a.358.358 0 01.222.61L8.54 7.373l.873 3.172a.361.361 0 01-.31.452l-.051.002a.36.36 0 01-.15-.039L5.998 9.524 3.1 10.959a.361.361 0 01-.099.033l-.05.005a.361.361 0 01-.362-.454l.873-3.172-2.357-2.335a.358.358 0 01.222-.61l2.862-.284 1.485-2.944A.364.364 0 015.999 1z" fill="currentColor" fill-rule="nonzero"></path></svg><svg aria-hidden="true" width="12" height="12"><path d="M5.999 1c.136 0 .262.076.324.198L7.81 4.144l2.862.284a.358.358 0 01.222.61L8.54 7.373l.873 3.172a.361.361 0 01-.31.452l-.051.002a.36.36 0 01-.15-.039L5.998 9.524 3.1 10.959a.361.361 0 01-.099.033l-.05.005a.361.361 0 01-.362-.454l.873-3.172-2.357-2.335a.358.358 0 01.222-.61l2.862-.284 1.485-2.944A.364.364 0 015.999 1z" fill="currentColor" fill-rule="nonzero"></path></svg><svg aria-hidden="true" width="12" height="12"><path d="M2.712 10.917a.361.361 0 01-.122-.373l.388-1.41.483-1.762-2.355-2.334a.358.358 0 01.171-.603l.051-.008 2.862-.283L5.675 1.2a.364.364 0 01.274-.196L6.001 1v.002l.049.003a.364.364 0 01.273.195L7.81 4.146l2.862.283a.358.358 0 01.222.61L8.54 7.375l.873 3.172a.361.361 0 01-.511.416L6 9.525 3.103 10.96a.36.36 0 01-.391-.043zM6 2.041v6.817c.078 0 .156.015.23.042l.064.028 2.292 1.135-.69-2.512a.667.667 0 01.129-.602L8.07 6.9l1.889-1.873-2.215-.219a.667.667 0 01-.495-.304l-.034-.059z" fill="currentColor" fill-rule="nonzero"></path></svg><span class="sr-only">4.6 out of 5 stars ; 10645 reviews</span><span class="Text-ds Text-ds--body-3 Text-ds--left Text-ds--neutral-600" aria-hidden="true">(10,645)</span></div></div><div class="ProductCard__price"><div class="ProductPricing"><span class="Text-ds Text-ds--body-2 Text-ds--left Text-ds--black">$52.00 </span></div></div><div class="ProductCard__offers"><div><p class="Text-ds Text-ds--body-3 Text-ds--left Text-ds--magenta-500">Bundle &amp; Save!</p></div></div></div></a>

def get_product_links(): 
    r = requests.get("https://www.ulta.com/shop/makeup/face/foundation")
    soup = BeautifulSoup(r.content, "html.parser")
    content = soup.find_all(class_='ProductCard')
    product_links = []
    for c in content:
        product_link_tag = c.find('a', class_='Link_Huge Link_Huge--secondary')
        product_link = product_link_tag.get('href')
        product_name = product_link_tag.get_text()
        #print(product_name)
        #print(product_link)

        #print(get_product_details(product_link) )




def extract_text_fields(json_data):
    """Extract all text fields using jsonpath-ng."""
    jsonpath_expression = parse('$.._value')
    matches = jsonpath_expression.find(json_data)
    return [match.value for match in matches]

def html_to_text_json(html_content):
    """Convert HTML content to a nested JSON containing only text fields."""
    json_data = html_to_json.convert(html_content)
    text_fields = extract_text_fields(json_data)
    #print(json.dumps(json_data, indent=4))
    return text_fields

# Example HTML content
r = requests.get('https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420')
soup = BeautifulSoup(r.content, "html.parser")
html_content =  soup.find_all(class_='ProductHero__content')
#print(html_content)


# Convert HTML to text-only JSON
text_json = html_to_text_json(str(html_content))

# Print the JSON output
#print(json.dumps(text_json, indent=4) )
print(' '.join(text_json))

## Get Specific Product Fields

In [ ]:
import requests
from bs4 import BeautifulSoup
import html_to_json
import json
from jsonpath_ng import jsonpath, parse
#pr_review
def get_product_reviews(product_link):
    r = requests.get('https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420')
    soup = BeautifulSoup(r.content, "html.parser")
    content = soup.find_all(class_='ProductHero__content')
    print(content)
    reviews = []
 
    for c in content:
        name = c.find(class_= 'Text-ds Text-ds--body-1 Text-ds--left')
        price = c.find(class_='ProductPricing')
        id_num = c.find(class_='Text-ds Text-ds--body-3 Text-ds--left Text-ds--neutral-600')

        review_content = soup.find_all(class_='reviews')


        #product_link = product_link_tag.get('href')
        product_price = price.get_text()
        product_name = name.get_text()
        product_id = id_num.get_text()
        #print(product_name, product_price, product_id)



  

print( get_product_reviews('') )  

## Get Review Info

In [ ]:
import requests
from bs4 import BeautifulSoup
import html_to_json
import json
from jsonpath_ng import jsonpath, parse
#pr_review


def extract_text_fields(json_data):
    """Extract all text fields using jsonpath-ng."""
    jsonpath_expression = parse('$.._value')
    matches = jsonpath_expression.find(json_data)
    return [match.value for match in matches]

def html_to_text_json(html_content):
    """Convert HTML content to a nested JSON containing only text fields."""
    json_data = html_to_json.convert(html_content)
    text_fields = extract_text_fields(json_data)
    #print(json.dumps(json_data, indent=4))
    return text_fields


# Example HTML content
r = requests.get('https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420')
soup = BeautifulSoup(r.content, "html.parser")
html_content =  soup.find_all(class_='ProductReviews__Content')
#print(str(html_content))


# Convert HTML to text-only JSON
text_json = html_to_text_json(str(html_content))

# Print the JSON output
#print(json.dumps(text_json, indent=4) )
print(text_json)

In [ ]:
import requests
from bs4 import BeautifulSoup
import html_to_json
import json
from jsonpath_ng import jsonpath, parse
from requests_html import HTMLSession  
import nest_asyncio
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

'''
#&pr_rd_page=10
r = requests.get('https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420&pr_rd_page=10')
soup = BeautifulSoup(r.content, "html.parser")
html_content =  soup.find_all(class_='ProductReviews__Content')
print(html_content)
'''
url = 'https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420'
# Setup WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get('file://' + 'temp.html')


with open('temp.html', 'w') as file:
    file.write(html_content)

# Extract the JavaScript code from a specific part of the HTML
soup = BeautifulSoup(html_content, 'html.parser')
content_div = soup.find('div', {'id': 'content'})
scripts = content_div.find_all('script')

javascript_code = []
for script in scripts:
    if script.string:
        javascript_code.append(script.string.strip())

# Execute the JavaScript code
for js in javascript_code:
    driver.execute_script(js)

# Optionally, print the result of JavaScript execution
result = driver.find_element_by_id('result')

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By

#from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



# Set up Chrome options
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode (no GUI)
options.add_argument('--disable-gpu')  # Disable GPU hardware acceleration
options.add_argument('--no-sandbox')  # Bypass OS security model

# Initialize the WebDriver
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
# Define a wait time
wait = WebDriverWait(driver, 10)

try:
    # Wait for the product heading to be present
    product_heading = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ProductReviews__Content')))
    print(product_heading.text)

    # If you need to extract reviews, wait for the review section to be present
    reviews = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'p-w-r')))
    for review in reviews:
        print(review.text)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the WebDriver
    driver.quit()


In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By

# Set up Chrome options
options = webdriver.ChromeOptions()
# Uncomment the following line to run in headless mode
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize the WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

try:
    # Open the URL
    url = "https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420"
    driver.get(url)

    # Print the title of the page to verify it worked
    print(driver.title)
    
    # Wait for the reviews section to load
    driver.implicitly_wait(10)

    # Find and print reviews
    reviews = driver.find_elements(By.CLASS_NAME, "ProductReviews__Reviews--containerList")
    for review in reviews:
        print(review.text)
        print(review.text, "works")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the WebDriver
    driver.quit()

In [ ]:


import time


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up Chrome options
options = webdriver.ChromeOptions()
# Uncomment the following line to run in headless mode
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize the WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)


# Navigate to the Ulta product page
driver.get("https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420")

# Define variables for pagination
current_page = 1
max_pages = 5  # Example: scrape up to 5 pages

#review_elements = driver.find_element('xpath', "//div[contains(@class, 'ProductReviews__Reviews--containerList')]")
#print(review_elements) 
articleDictionary = dict()
myKey = ""
myValue_total = ""

# Optionally wait for the page to load completely
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='ProductReviews__Reviews--containerList']")))

# Get the HTML content
html_content = driver.page_source
print(html_content) 


while current_page <= max_pages:
    # Scrape data from the current page
    review_elements = driver.find_elements('xpath', "//div[contains(@class, 'ProductReviews')]")
    #print(review.get_attribute("innerText"))
    
    #scripts = driver.execute_script()
    
    for review in review_elements:
        #print(review.tag_name)
        #print('NEW LINE')
        '''
        # Extract review information
        #rating = review.find_element_by_xpath(".//div[contains(@class, 'Rating')]//svg").get_attribute("aria-label").strip()
        review_text = review.find_element('xpath', ".//div[contains(@class, 'p-w-r')]").text.strip()
        
        # Process the data (print for now)
        #print(f"Rating: {rating}")
        print(f"Review Text: {review_text}")
        print("-" * 50)
        '''
    
    
    # Navigate to the next page if available
    try:
        next_button = driver.find_element('xpath',"//a[contains(@class, 'pr-rd-pagination')]")
        if next_button:
            next_button.click()
            current_page += 1
            time.sleep(2)  # Add a short delay to ensure the next page loads completely
    except NoSuchElementException:
        break  # Exit loop if no next page link found or reached max pages

# Close the WebDriver session
driver.quit()


In [ ]:
import requests
from bs4 import BeautifulSoup
import html_to_json
import json
from jsonpath_ng import jsonpath, parse

# Load the HTML content from a file
r = requests.get("https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420")


# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(r.content, 'html.parser')

# Specify the class name of the div you want to extract information from
target_class = 'p-w-r'

# Find all div elements with the specified class
div_elements = soup.find_all('div', class_=target_class)
print(div_elements)

# Extract the desired information from each div element
for index, div in enumerate(div_elements):
    # Extract text content of the div
    div_text = div.get_text(strip=True)
    print(f"Div {index + 1} text:\n{div_text}\n")
    
    # Extract other specific elements or attributes within the div if needed
    # Example: Extract all href attributes of <a> tags within the div
    '''
    links = div.find_all('a')
    for link in links:
        print(f"Link: {link['href']}")
    '''
'''
# Optionally, you can write the extracted information to a file
with open('extracted_div_content.txt', 'w', encoding='utf-8') as file:
    for div in div_elements:
        div_text = div.get_text(strip=True)
        file.write(f"Div text:\n{div_text}\n\n")
        links = div.find_all('a')
        for link in links:
            file.write(f"Link: {link['href']}\n")
'''



In [ ]:
from bs4 import BeautifulSoup

# Load the HTML content from a file
with open('page_source.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find all script tags
script_tags = soup.find_all('script')

# Extract the JavaScript code from each script tag
javascript_code = []
for script in script_tags:
    if script.string:
        javascript_code.append(script.string)

# Print all JavaScript code found
for index, code in enumerate(javascript_code):
    print(f"JavaScript code {index + 1}:\n{code}\n")

'''
# Optionally, you can write the JavaScript code to a file
with open('extracted_javascript.js', 'w', encoding='utf-8') as file:
    for code in javascript_code:
        file.write(code + '\n\n')
'''

In [ ]:
import requests
from bs4 import BeautifulSoup
import html_to_json
import json
from jsonpath_ng import jsonpath, parse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Set up ChromeOptions
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode
chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration (often needed in headless mode)
# Set up Chrome WebDriver (make sure chromedriver is in your PATH)
driver = webdriver.Chrome(chrome_options)


# Load the HTML content from a file
driver.get("https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420&pr_rd_page=2")
# Wait for pr-review elements to be present

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "ProductReviews__Reviews--containerList"))
)

# Wait for the pagination link to be clickable
next_page_link = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.pr-rd-pagination a.pr-rd-pagination-btn--next'))
)

# Click the pagination link to load more content
next_page_link.click()

try:
    pr_reviews = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "pr-review"))
    )

    print("got reviews")
finally:
    # Get the initial text content of the first pr-review element
    print([pr_review.text for pr_review in pr_reviews])

# Wait until the text content of pr-review elements changes
WebDriverWait(driver, 10).until(
    lambda driver: EC.text_to_be_present_in_element((By.CLASS_NAME, "pr-review"), initial_text)(driver) is False
)
file_path = "webtext_page2_with_seleniumv2.json"
with open(file_path, 'w') as file:
    file.write(str(driver.page_source))

In [ ]:

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up Chrome options
options = webdriver.ChromeOptions()
# Uncomment the following line to run in headless mode
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize the WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# Function to extract the XHR URL
def extract_xhr_url(url):
    driver.get(url)

    # Wait until the desired network request is made
    def wait_for_xhr_request(driver):
        for request in driver.requests:
            if request.response and 'displaypowerreviews' in request.url:
                return request.url
        return False

    # WebDriverWait to wait for the XHR request
    xhr_url = WebDriverWait(driver, 100).until(wait_for_xhr_request)
    print('XHR URL:', xhr_url)

# Example URL of a product page on Ulta
product_url = 'https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420'
extract_xhr_url(product_url)

driver.quit()


In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Set up Chrome options
options = webdriver.ChromeOptions()
# Uncomment the following line to run in headless mode
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize the WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# Function to extract the XHR URL
def extract_xhr_url(url):
    driver.get(url)

     # Wait for XHR request that contains 'reviews' in URL
    reviews_request = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "ProductReviews__Reviews--containerList"))
    )
        
    with open('get_requests.txt', 'w', encoding='utf-8') as file:
        for request in driver.requests:
            file.write(request.url + '\n\n')
    # Loop to wait for the XHR request indefinitely
    #xhr_url =  reviews_request.path

    #print('XHR URL:', xhr_url)

# Example URL of a product page on Ulta
product_url = 'https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420'
extract_xhr_url(product_url)

driver.quit()


In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

from selenium.webdriver.common.action_chains import ActionChains
import time

# Set up Chrome options
options = webdriver.ChromeOptions()
# Uncomment the following line to run in headless mode
#options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize the WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.get('https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420')

# Function to scroll down
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    print('def scroll_down')

def have_reviews():
    #actions = ActionChains(driver)
    #actions.move_to_element(element).perform()
    print('found the element!')
    return any(['display.powerreviews' in request.url for request in driver.requests])

# Scroll down and load more content
while not have_reviews():
    scroll_down(driver)
    

# Function to extract the XHR URL
def extract_xhr_url():
    with open('get_requests.txt', 'w', encoding='utf-8') as file:
        for request in driver.requests:
            file.write(request.url + '\n\n')
# Close the WebDriver
extract_xhr_url()
driver.quit()


In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from seleniumwire import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time

# Set up Chrome options
options = webdriver.ChromeOptions()
# Uncomment the following line to run in headless mode
#options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize the WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.get('https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420')
section_locator = (By.ID, "reviews")
print(selection_locator)

# Wait for the section to be visible
section_element = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located(section_locator)
)
# Scroll to the specific section
driver.execute_script("arguments[0].scrollIntoView();", section_element)

# Function to check if at bottom of the page
def is_at_bottom():
    # Implement logic to check if at the bottom
    return driver.execute_script("return (window.innerHeight + window.scrollY) >= document.body.offsetHeight")



# Function to extract the XHR URL
def extract_xhr_url():
    with open('zara_request.txt', 'w', encoding='utf-8') as file:
        for request in driver.requests:
            file.write(request.url + '\n\n')

# Scroll down to the bottom of the page
while not is_at_bottom():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    WebDriverWait(driver, 10).until(
        lambda driver: driver.execute_script('return document.readyState') == 'complete')

# Close the WebDriver
extract_xhr_url()
driver.quit()


In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from seleniumwire import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time

# Set up Chrome options
options = webdriver.ChromeOptions()
# Uncomment the following line to run in headless mode
#options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize the WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.get('https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420')

def load():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    WebDriverWait(driver, 30).until(
        lambda driver: driver.execute_script('return document.readyState') == 'complete' )

# Function to extract the XHR URL
def extract_xhr_url():
    with open('zara_request.txt', 'w', encoding='utf-8') as file:
        for request in driver.requests:
            file.write(request.url + '\n\n')

# Function to check if at bottom of the page
def is_at_bottom():
    # Slow scroll to the section
    scroll_height = driver.execute_script('return document.body.scrollHeight')
    current_position = 0
    while current_position < scroll_height:
        driver.execute_script('window.scrollTo(0, {});'.format(current_position))
        current_position += 100  # Adjust scrolling speed by changing increment
        time.sleep(0.1)  # Adjust sleep time to control scrolling speed

    # Pause for a few seconds to observe the section
    time.sleep(10)  # Adjust as needed

# Function to extract the XHR URL
def extract_xhr_url():
    with open('zara_request.txt', 'w', encoding='utf-8') as file:
        for request in driver.requests:
            file.write(request.url + '\n\n')

load()

try:
    section_id = "reviews"
    section_element = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.ID, section_id))
    )

    # Scroll to the section using scrollIntoView
    driver.execute_script("arguments[0].scrollIntoView({ block: 'center', inline: 'center' });", section_element)

    # Pause for a few seconds to observe the section
    scroll_height = driver.execute_script('return document.body.scrollHeight')
    current_position = 0
    while current_position < scroll_height:
        driver.execute_script('window.scrollTo(0, {});'.format(current_position))
        current_position += 100  # Adjust scrolling speed by changing increment
        time.sleep(0.2)  # Adjust sleep time to control scrolling speed
    extract_xhr_url()
    time.sleep(5)

    
#time.sleep(5)  # Adjust as needed

finally:
    # Close the browser
    driver.quit()

'''
    # Pause for a few seconds to observe the section
    time.sleep(5)  # Adjust as needed
    # Scroll down to the bottom of the page
    while not is_at_bottom():
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 15).until(
            lambda driver: driver.execute_script('return document.readyState') == 'complete')
        
'''


In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from seleniumwire import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.common.by import By
#import logging

# Set up Chrome options
options = webdriver.ChromeOptions()
# Uncomment the following line to run in headless mode
#logging.basicConfig(level=logging.INFO)
#options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
#options.add_experimental_option('excludeSwitches', ['enable-logging'])


# Initialize the WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.maximize_window()
driver.get('https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420')
xhr_links = [] #stores the specific xhr links needed


# Function to extract the XHR URL
def extract_xhr_url():
    for request in driver.requests: 
        xhr_links.append(request.url)

 
# Define a function to gradually scroll through the entire page
def slow_scroll(start_position):
    scroll_height = driver.execute_script('return document.body.scrollHeight')
    current_position = start_position
    while current_position < scroll_height:
        driver.execute_script('window.scrollTo(0, {});'.format(current_position))
        current_position += 100  # Adjust scrolling speed by changing increment
        time.sleep(0.2)  # Adjust sleep time to control scrolling speed

try:
    # Wait for the section to be visible (adjust timeout as needed)
    section_id = "reviews"
    section_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.ID, section_id))
    )
    height =  0
    time.sleep(2)
    slow_scroll(height)
    extract_xhr_url()
finally:
    # Close the browser
    driver.quit()

for x in xhr_links: 
    if 'display.powerreviews' in x: 
        print(x, "\n")


In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from seleniumwire import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Set up Chrome options
options = webdriver.ChromeOptions()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_experimental_option('excludeSwitches', ['enable-logging'])

# Initialize the WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.maximize_window()
driver.get('https://www.ulta.com/p/double-wear-stay-in-place-foundation-xlsImpprod14641507?sku=2309420')
xhr_links = []  # Stores the specific xhr links needed

# Function to extract the XHR URL
def extract_xhr_url():
    for request in driver.requests:
        xhr_links.append(request.url)

# Define a function to gradually scroll through the entire page
def slow_scroll(start_position):
    scroll_height = driver.execute_script('return document.body.scrollHeight')
    current_position = start_position
    while current_position < scroll_height:
        driver.execute_script('window.scrollTo(0, {});'.format(current_position))
        current_position += 100  # Adjust scrolling speed by changing increment
        time.sleep(0.2)  # Adjust sleep time to control scrolling speed

# Function to click the "Next" link
def click_next_page():
    try:
        next_page_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'pr-rd-pagination-btn--next'))
        )
        try:
            next_page_link.click()
        except ElementClickInterceptedException:
            driver.execute_script("arguments[0].click();", next_page_link)
        return True
    except NoSuchElementException:
        return False

try:
    # Wait for the section to be visible (adjust timeout as needed)
    section_id = "reviews"
    section_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.ID, section_id))
    )
    height = section_element.location['y']
    time.sleep(2)
    
    # Scroll to the reviews section
    driver.execute_script("arguments[0].scrollIntoView({ behavior: 'instant', block: 'start' });", section_element)
    
    # Start the slow scroll from the reviews section
    slow_scroll(height)
    
    # Extract initial XHR URLs
    extract_xhr_url()
    
    # Click the "Next" link and scroll again
    pg_count = 1
    while click_next_page() and pg_count < 2:
        pg_count += 1
        time.sleep(2)  # Wait for new content to load
        slow_scroll(height)
        extract_xhr_url()

finally:
    # Close the browser
    driver.quit()

# Print the extracted XHR URLs
for x in xhr_links: 
    if 'display.powerreviews' in x: 
        print(x, "\n")
